# Install Playwright CLI

You can install [Playwright](https://playwright.dev/) as a dotnet global tool or as npm global package. In this example we will install Playwright as a global tool.

In [ ]:
#!powershell
dotnet tool install --global Microsoft.Playwright.CLI

# Save session state

Any application that requires login, still save the state (logged in or not) using cookies. If you start a new session, with the previously captured session state, the application consideres you as having "logged in", even though you did not enter your credentials.

The code block below creates an empty project, installs the browsers (Chromium, Firefox and Webkit) and opens Chromium in "automation" mode. In this new browser window, navigate to the Power Apps application, login with you credentials, and close the window after you have logged in.



In [ ]:
#!powershell
# Create project
dotnet new console -n PlaywrightDemo
cd PlaywrightDemo

# Install dependencies, build project and download necessary browsers.
dotnet add package Microsoft.Playwright
dotnet build
playwright install

playwright codegen --save-storage=auth.json

Move-Item .\auth.json ../. -Force
cd..
Remove-Item ./PlaywrightDemo -Force -Recurse


After running the commands the session state will be captured in a file called `auth.json`. There are only two cookies needed for Power Apps to consider you as "logged in". They are

1. _CrmOwinAuth_
2. _ESTSAUTHPERSISTENT_

You can remove all the other information except these two. Below is a sample for `auth.json`.

```json
{
  "cookies": [
    {
      "sameSite": "None",
      "name": "CrmOwinAuth",
      "value": "REMOVED",
      "domain": ".crm.dynamics.com",
      "path": "/",
      "expires": 1953359538.945023,
      "httpOnly": true,
      "secure": true
    },
    {
      "sameSite": "None",
      "name": "ESTSAUTHPERSISTENT",
      "value": "REMOVED",
      "path": "/",
      "expires": 1645602747.992615,
      "httpOnly": true,
      "secure": true
    }
  ]
}
```

Now we need to install the Playwright package on the Notebook, so that we can run some automation from this Notebook.

In [ ]:
#r "nuget:Microsoft.Playwright"

# Configuring screenshots

[screenshots.config.json](screenshots.config.json) is where you specify the title of the screeshot, file name of the screenshot and URL to navigate to. Below is an example config.

```json
[
    {
        "title": "Home",
        "fileName": "Home",
        "url": "https://make.powerapps.com"
    },
    {
        "title": "Customer Service App",
        "fileName": "Model Driven App",
        "url": "https://org.crm.dynamics.com/main.aspx?appid=fc4884b2-6119-e811-a94f-000d3ae060f6"
    }
]
```

The snippet below reads code from the config file.

In [ ]:
using Microsoft.Playwright;
using System.Threading.Tasks;
using System.IO;
using System.Text.Json;

record Config(string fileName, string url, string title, string description);
var config = JsonSerializer.Deserialize<Config[]>(File.ReadAllText("screenshots.config.json"));

# Running Playwright

Since we now have the `auth.json` file with the persisted cookies we can start a new browser session with the pre-save cookie information. It will navigate us straight to the URLs mentioned in the config file, with no login screen at all. The cookies are valid for approximately 2 weeks. So, you might get error after the cookie expires. If that is the case, you need to run [Save Session State](#save-session-state) step again, to create a new `auth.json`. `auth.json` contains sensitive information, so it is ignored on `.gitignore`, so <ins>do not</ins> commit this to source control.

In [ ]:
using (var playwright = await Playwright.CreateAsync())
{
        //Use this if you are behind a proxy
        // var browser = await playwright.Firefox.LaunchAsync(new BrowserTypeLaunchOptions
        // {
        //     Headless = false,
        //     Proxy = new Proxy{Server=""}
        // });
        var browser = await playwright.Chromium.LaunchAsync(new BrowserTypeLaunchOptions{Timeout=60000});        
        var context = await browser.NewContextAsync(new BrowserNewContextOptions { 
            StorageStatePath = "../auth.json",
            RecordVideoDir = "../videos",
            RecordVideoSize = new RecordVideoSize() { Width = 1920, Height = 1080 }
        });
        var page = await context.NewPageAsync();
        await page.SetViewportSizeAsync(1920, 1080);
        foreach (var c in config)
        {
            await page.GotoAsync(c.url, new PageGotoOptions { WaitUntil = WaitUntilState.NetworkIdle });
            await page.ScreenshotAsync(new PageScreenshotOptions { Path = $"../screenshots/{c.fileName}.png" });            
        }
        await context.CloseAsync();
}

# Generate screenshot documentation

We can dynamically add the screenshot document and text using the Notebook itself by running the snippet below.

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

var screenshot = div(
    h1(summary($"Screenshots as of {DateTime.Now.ToString()}")),
    details(
        config.Select(c => 
            div(
                h2(c.title),
                div(c.description),
                br,
                img[src:$"../screenshots/{c.fileName}.png",width:960, height: 540]
            )
        )
    )
);
display(screenshot);

//START: Remove this if you don't want the screenshot HTML to be saved into a file
if(!Directory.Exists("../documentation")) Directory.CreateDirectory("../documentation");

File.WriteAllText($"../documentation/{DateTime.Now.ToString("yyyy_MM_dd_HH_mm")}.html", @$"<!DOCTYPE html>
<html lang=""en"">
<head>
    <meta charset=""UTF-8"">
    <meta http-equiv=""X-UA-Compatible"" content=""IE=edge"">
    <meta name=""viewport"" content=""width=device-width, initial-scale=1.0"">
</head>
<body>
    {screenshot}
</body>
</html>");
//END